In [1]:
import os
import sys
import numpy as np
sys.path.insert(0, os.path.realpath('../'))

In [2]:
import LDAQ

In [3]:
# here we should show how to use the visualization object:
# - use of functions on channels of subplot
    # - use of built in functions
    # - use of custom functions
# - refresh rates, t_span, x_lim, y_lim
# - use case add_image, add_lines
# ...

In [4]:
# Create simulated data acquisition sources:
acq_simulated_data = LDAQ.simulator.SimulatedAcquisition(acquisition_name='sim_data', multi_processing=False)

In [5]:
# define simulated data using numpy array (array will be looped over):
fs = 10000
t = np.arange(fs*2)
x = np.sin(2*np.pi*50*t/fs) + np.random.randn(len(t))
y = np.sin(2*np.pi*100*t/fs + np.pi/2)
simulated_data = np.array([x, y]).T

# define simulated data using a function:
def simulate_signal(t, f1, f2):
    """
    first argument must be time.
    If multi_processing is True, then the function must be pickable and 
    all libraries must be imported inside the function.
    """
    import numpy as np
    sig1 = np.sin(2*np.pi*f1*t) + np.random.rand(len(t))*0.3
    sig2 = np.cos(2*np.pi*f2*t) + np.random.rand(len(t))*0.3
    return np.array([sig1, sig2]).T

In [6]:
# set simulated data (first argument is the function or array):
acq_simulated_data.set_simulated_data(simulated_data, channel_names=["ch1", "ch2"], sample_rate=fs, args=(84, 120))

In [7]:
# create simulated video acquisition source:
acq_simulated_video = LDAQ.simulator.SimulatedAcquisition(acquisition_name='sim_video', multi_processing=False)

In [8]:
# define simulated video using numpy array:
fs = 30
simulated_video = np.random.randn(1000, 100, 100)

# define simulated video using a function:
def simulate_video(t):
    """
    first argument must be time.
    If multi_processing is True, then the function must be pickable and 
    all libraries must be imported inside the function.
    """
    import numpy as np
    N = t.shape[0]	
    return np.random.randn(N, 100, 100)

In [9]:
acq_simulated_video.set_simulated_video(simulate_video, channel_name_video='cam', sample_rate=fs)

In [10]:
def virtual_channel_func(self, ch1):
    try:
        # At virtual channel definition, retrieving data or channel indices is not yet possible for all channels.
        # Therefore, we use a try-except block to properly execute channel definition.
        time, data = self.get_data(N_points=2)
        i_ch3 = self.get_channel_index("ch3", channel_type='data') # retrieve data index channel of ch1
        ch3_prev = data[-1, i_ch3] # retrieve last value of ch1
        
    except:
        # at channel definition, no data is available yet. Therefore, we set the previous value to 0.
        ch3_prev = 0
        
    # cumulative sum of ch1:
    ch1_cumsum = np.cumsum(ch1) + ch3_prev	
    return ch1_cumsum.reshape(-1,1)

acq_simulated_data.add_virtual_channel("ch3", ["ch1"], function=virtual_channel_func)

In [11]:
def custom_vis_function(self, data):
    # this function is applied to each channel of the subplot individually.
    # data is a 1D array.
    
    return data**3 + 0.5

vis = LDAQ.Visualization(sequential_plot_updates=False)
vis.add_lines((0,0), source='sim_data', channels=["ch1", "ch2"], refresh_rate=50)
vis.add_lines((0,1), source='sim_data', channels=["ch1", "ch2"], refresh_rate=500, function="fft")
vis.add_lines((1,0), source='sim_data', channels=["ch1", "ch2"], refresh_rate=50, function=custom_vis_function)
vis.add_lines((1,0), source='sim_data', channels=["ch3"], refresh_rate=50)
vis.add_lines((2, 0), source='sim_data', channels=[("ch1", "ch2")])

vis.add_image(source="sim_video", channel="cam", refresh_rate=30)

vis.config_subplot((0,0), t_span=0.04, ylim=(-1.1, 1.1))
vis.config_subplot((0,1), t_span=0.5, ylim=(0, 1.1), xlim=(0, 500), rowspan=2)
vis.config_subplot((1,0), t_span=0.04)

In [16]:
ldaq = LDAQ.Core(acquisitions=[acq_simulated_data, acq_simulated_video], visualization=vis)
ldaq.run(3.)

In [20]:
ldaq.get_measurement_dict()

{'sim_data': {'time': array([0.0000e+00, 1.0000e-04, 2.0000e-04, ..., 2.9997e+00, 2.9998e+00,
         2.9999e+00]),
  'channel_names': ['ch1', 'ch2', 'ch3'],
  'data': array([[ 1.18907369e+00, -8.76306680e-01,  3.53242568e+02],
         [-1.27870428e-01, -9.04827052e-01,  3.53114697e+02],
         [-9.83610267e-01, -9.29776486e-01,  3.52131087e+02],
         ...,
         [ 3.26830815e-02, -7.70513243e-01,  7.62472231e+02],
         [-1.99687628e+00, -8.09016994e-01,  7.60475355e+02],
         [ 1.53863497e-02, -8.44327926e-01,  7.60490741e+02]]),
  'sample_rate': 10000},
 'sim_video': {'time': array([0.        , 0.03333333, 0.06666667, 0.1       , 0.13333333,
         0.16666667, 0.2       , 0.23333333, 0.26666667, 0.3       ,
         0.33333333, 0.36666667, 0.4       , 0.43333333, 0.46666667,
         0.5       , 0.53333333, 0.56666667, 0.6       , 0.63333333,
         0.66666667, 0.7       , 0.73333333, 0.76666667, 0.8       ,
         0.83333333, 0.86666667, 0.9       , 0.9333333